In [2]:
import torch

import Code.LateResidualPyTorch.LateResidualNeuralNetwork as LRNN

In [3]:
torch_model = LRNN.NNet(n_inputs=1, n_outputs=1, n_hiddens_list=[2,2], optimizer='Adam')
torch_model.load_state_dict(torch.load('model.pt'))
torch_model.model.eval()

Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=2, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2, out_features=1, bias=True)
)

In [4]:
# see layers
example_input = torch.rand(2, 1)
traced_model = torch.jit.trace(torch_model.model, example_input)
out = traced_model(example_input)

#### CoreML Tools

In [7]:
import coremltools as ct

model = ct.convert(
    traced_model,
    convert_to="mlprogram", 
    inputs=[ct.TensorType(name="input", shape=example_input.shape)],
)
model.save("lrnn_test.mlpackage")

Running MIL Common passes:   0%|          | 0/38 [00:00<?, ? passes/s]/Users/matthewernst/miniforge3/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '15', of the source model, has been renamed to 'var_15' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Clean up passes: 100%|██████████| 11/11 [00:00<00:00, 2497.15 passes/s]


In [10]:
model = ct.convert(
    traced_model,
    inputs=[ct.TensorType(name="input", shape=example_input.shape)]
)
model.save("lrnn_test.mlmodel")

Running MIL Common passes:   0%|          | 0/38 [00:00<?, ? passes/s]/Users/matthewernst/miniforge3/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '15', of the source model, has been renamed to 'var_15' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 11/11 [00:00<00:00, 53461.58 ops/s]
